# CitiBike Data Exploration

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the various `2023$$citibike-tripdata_#.csv` files from the Terminal. Name the database `citi_bike` and the collection `rides_2023`.

Import the dataset with: `mongoimport --type csv -d citi_bike -c rides_2023 --headerline --file 202301-citibike-tripdata_1.csv` and similar patterns. Make sure you use the `--drop` parameter the first time through, but not for any of the later files as we don't want to overwrite existing data except with the first file.  The rest should just append.

In [41]:
# Import dependencies
from pprint import pprint
import pandas as pd
import datetime
from math import radians, sin, cos, sqrt, atan2
import pymongo

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)  # Using the default port

In [3]:
# assign the citi_bike database to a variable name
db = mongo['citi_bike']

In [4]:
# review the collections in our new database
db.list_collection_names() # To begin with, the only collection should be 'rides_2023' and it will be empty

['rides_sample', 'rides_2023']

In [5]:
# review a document in the rides_sample or rides_2023 collection
pprint(db.rides_2023.find_one(), indent=2) # This prints out a single document from within the collection. 

{ '_id': '4A86C1475DCCADA0',
  'end_lat': 40.7770575,
  'end_lng': -73.97898475,
  'end_station_id': 7175.05,
  'end_station_name': 'Columbus Ave & W 72 St',
  'ended_at': '2023-01-26 11:05:17',
  'member_casual': 'member',
  'rideable_type': 'classic_bike',
  'start_lat': 40.75972414,
  'start_lng': -73.973664165,
  'start_station_id': 6659.03,
  'start_station_name': 'E 53 St & Madison Ave',
  'started_at': '2023-01-26 10:53:44'}


In [6]:
# This is just a lot of data to work with. There are 36,000,000 records in just the one year of 2023!
# So, let's cut that down by 99%... We will just take one record out of every hundred.  That will still leave 320,000 records.
# We'll try it out using just a single sample data file

# Connect to MongoDB
source_collection = db['rides_2023']
new_collection = db['reduced_rides']

# Get every hundredth record from the source collection
selected_documents = []
counter = 0

for doc in source_collection.find():
    counter += 1
    if counter % 100 == 0:
        selected_documents.append(doc)

# Create a new collection and insert the selected documents
if selected_documents:
    new_collection.insert_many(selected_documents)
    print(f"{len(selected_documents)} documents inserted into the new collection.")
else:
    print("No documents selected to insert into the new collection.")

# This gives us a much smaller dataset to analyze!  For anything where we are using 'sum' or 'total' or the like, we will just multiply by 100...

362263 documents inserted into the new collection.


In [7]:
rides=db['reduced_rides']

In [43]:
# Some of the data we created are not accurate (incomplete), so we need to drop some records

rides.delete_many({'end_lat': ''})
rides.delete_many({'end_lng': ''})
rides.delete_many({'end_station_id': ''})
rides.delete_many({'ended_at': ''})
rides.delete_many({'member_casual': ''})
rides.delete_many({'rideable_type': ''})

# There are also some records where the ride clearly ended, but the bike didn't get re-docked.  
# We need to eliminate those from our dataset in order to avoid skewing averages
# Any ride where the duration is greater than 10 hours (600 minutes) and the distance is
# less than 10km will be assumed to be faulty.

query = {'ride_duration': {'$gt': 600}, 'distance': {'$lt': 10}}
result = rides.delete_many(query)


In [9]:
# Now we need to do a little cleanup on data formating... Our dates came in as srings.

update_query = [
    {
        '$set': {
            'started_at': {
                '$toDate': '$started_at'
            }
        }
    }
]
rides.update_many({}, update_query)


In [10]:
# Unfortunately, the started_at and ended_at fields are strings, not dates.  We need to convert them.

update_query = [
    {
        '$set': {
            'ended_at': {
                '$toDate': '$ended_at'
            }
        }
    }
]
rides.update_many({}, update_query)


In [11]:
pprint(rides.find_one(), indent=2)

{ '_id': '219BF53DA8A2E0C0',
  'end_lat': 40.877964,
  'end_lng': -73.8847550291,
  'end_station_id': 8795.03,
  'end_station_name': 'Grand Concourse & E Mosholu Pkwy S',
  'ended_at': datetime.datetime(2023, 1, 28, 20, 35, 31),
  'member_casual': 'member',
  'rideable_type': 'classic_bike',
  'start_lat': 40.860086441,
  'start_lng': -73.900726914,
  'start_station_id': 8550.04,
  'start_station_name': 'Morris Ave & E 184 St',
  'started_at': datetime.datetime(2023, 1, 28, 20, 16, 37)}


## Part 2: Calculated Fields
We now have all our data with the appropriate date types, but we need to perform some calculations to derive field values we will need:
* Duration of ride in minutes
* Distance of ride in km
* Month in which the ride took place (to allow summarization by month)

In [12]:
# assign the collection to a variable
rides = db['reduced_rides']

In [13]:
# Let's start by determining the duration of each ride... We can just subtract the datetimes
# and that will give us the duration in milliseconds.

pipeline = [
    {
        '$addFields': {
            'ride_duration': {
                '$subtract': ['$ended_at', '$started_at']
            }
        }
    }
]

# Perform the aggregation
aggregated_result = list(rides.aggregate(pipeline))

# Insert aggregated data back into a new collection
calculated_duration = db.calculated_duration

for result in aggregated_result:
    calculated_duration.insert_one(result)

In [14]:
# Only, we don't want to work with milliseconds.  I suspect minutes would be a good unit.
# So we will need to divide the ride duration by 60,000 (number of milliseconds in a minute)
# to get our tide duration in minutes.

# First, we will re-set our rides variable to be looking at our new collection...
rides = db['calculated_duration']

In [15]:
pipeline = [
    {
        '$addFields': {
            'ride_duration_minutes': { '$divide': ['$ride_duration', 60000] }
        }
    }
]

aggregated_result = list(rides.aggregate(pipeline))
calculated_durations = db.calculated_durations

for result in aggregated_result:
    calculated_durations.insert_one(result)

In [16]:
# Now we can drop a couple of our collections that we no longer need since they are duplicates...

db.reduced_rides.drop()
db.calculated_duration.drop()

In [17]:
# We can now get rid of the extra (millisecond) field we don't need...
# And rename the ride_duration_minutes field as just ride_duration

rides=db['calculated_durations']
rides.update_many(
    {},
    {'$unset': {'ride_duration': ""}}
)
rides.update_many(
    {},
    {'$rename': {'ride_duration_minutes': 'ride_duration'}}
)
# Let's check to see what we've got...

rides=db['calculated_durations']
pprint(rides.find_one(), indent=2)

{ '_id': '219BF53DA8A2E0C0',
  'end_lat': 40.877964,
  'end_lng': -73.8847550291,
  'end_station_id': 8795.03,
  'end_station_name': 'Grand Concourse & E Mosholu Pkwy S',
  'ended_at': datetime.datetime(2023, 1, 28, 20, 35, 31),
  'member_casual': 'member',
  'ride_duration': 18.9,
  'rideable_type': 'classic_bike',
  'start_lat': 40.860086441,
  'start_lng': -73.900726914,
  'start_station_id': 8550.04,
  'start_station_name': 'Morris Ave & E 184 St',
  'started_at': datetime.datetime(2023, 1, 28, 20, 16, 37)}


In [18]:
# We will use the Haversine formula to calculate distance between two points in kilometers

# We can call a javascript function in the middle of our aggregation!
pipeline = [
    {
        '$project': {
            '_id': '$_id',
            'rideable_type': '$rideable_type',
            'started_at': '$started_at',
            'ended_at': '$ended_at',
            'start_station_name': '$start_station_name',
            'start_station_id': '$start_station_id',
            'end_station_name': '$end_station_name',
            'end_station_id': '$end_station_id',
            'start_lat': '$start_lat',
            'start_lng': '$start_lng',
            'end_lat': '$end_lat',
            'end_lng': '$end_lng',
            'member_casual': '$member_casual',
            'ride_duration': '$ride_duration',
            'distance': {
                '$function': {
                    'body': f'''
                        function(lat1, lon1, lat2, lon2) {{
                            var R = 6371.0;
                            var dLat = (lat2 - lat1) * Math.PI / 180;
                            var dLon = (lon2 - lon1) * Math.PI / 180;
                            var a = Math.sin(dLat / 2) * Math.sin(dLat / 2) +
                                    Math.cos(lat1 * Math.PI / 180) * Math.cos(lat2 * Math.PI / 180) *
                                    Math.sin(dLon / 2) * Math.sin(dLon / 2);
                            var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));
                            var distance = R * c;
                            return distance;
                        }}
                    ''',
                    'args': ['$start_lat', '$start_lng', '$end_lat', '$end_lng'],
                    'lang': 'js'
                }
            }
        }
    }
]

    
aggregated_result = list(rides.aggregate(pipeline))
distances = db.distances

for result in aggregated_result:
    distances.insert_one(result)

In [19]:
# All right, let's see whether we now have distance (in km) in our new collection...

rides=db['distances']
db.calculated_durations.drop()
pprint(rides.find_one(), indent=2)

{ '_id': '219BF53DA8A2E0C0',
  'distance': 2.3990458227909364,
  'end_lat': 40.877964,
  'end_lng': -73.8847550291,
  'end_station_id': 8795.03,
  'end_station_name': 'Grand Concourse & E Mosholu Pkwy S',
  'ended_at': datetime.datetime(2023, 1, 28, 20, 35, 31),
  'member_casual': 'member',
  'ride_duration': 18.9,
  'rideable_type': 'classic_bike',
  'start_lat': 40.860086441,
  'start_lng': -73.900726914,
  'start_station_id': 8550.04,
  'start_station_name': 'Morris Ave & E 184 St',
  'started_at': datetime.datetime(2023, 1, 28, 20, 16, 37)}


In [20]:
# Next step is to pull out the month of the ride so we can prepare a summary dataset that has all rides by month, member_type, bicycle_type, ride_duration, and distance

pipeline = [
    {
        '$project': {
            '_id': '$_id',
            'rideable_type': '$rideable_type',
            'started_at': '$started_at',
            'ended_at': '$ended_at',
            'start_station_name': '$start_station_name',
            'start_station_id': '$start_station_id',
            'end_station_name': '$end_station_name',
            'end_station_id': '$end_station_id',
            'start_lat': '$start_lat',
            'start_lng': '$start_lng',
            'end_lat': '$end_lat',
            'end_lng': '$end_lng',
            'member_casual': '$member_casual',
            'ride_duration': '$ride_duration',
            'distance': '$distance',
            'month': {'$month': '$started_at' }}
    }
]

aggregated_result = list(rides.aggregate(pipeline))
full_data = db.full_data

for result in aggregated_result:
    full_data.insert_one(result)

In [21]:
rides=db['full_data']
db.distances.drop()
pprint(rides.find_one(), indent=2)

{ '_id': '219BF53DA8A2E0C0',
  'distance': 2.3990458227909364,
  'end_lat': 40.877964,
  'end_lng': -73.8847550291,
  'end_station_id': 8795.03,
  'end_station_name': 'Grand Concourse & E Mosholu Pkwy S',
  'ended_at': datetime.datetime(2023, 1, 28, 20, 35, 31),
  'member_casual': 'member',
  'month': 1,
  'ride_duration': 18.9,
  'rideable_type': 'classic_bike',
  'start_lat': 40.860086441,
  'start_lng': -73.900726914,
  'start_station_id': 8550.04,
  'start_station_name': 'Morris Ave & E 184 St',
  'started_at': datetime.datetime(2023, 1, 28, 20, 16, 37)}


## CSV Creation
Now we need to create the CSV's that we will be putting into Tableau for our analysis.  We will need the following CSV's
* A 'Summary' CSV that aggregates our data by month, rideable_type, member_type and provides count of trips, average duration, average distance


In [22]:
group_query = {
    '$group': {
        '_id': {
            'month': '$month',
            'member_casual': '$member_casual',
            'rideable_type': '$rideable_type'
        },
        'count': { '$sum': 1 },
        'avg_distance': {'$avg': '$distance'},
        'avg_duration': {'$avg': '$ride_duration'}
    }
}

aggregated_result = list(rides.aggregate([group_query]))


In [23]:
summary_df = pd.DataFrame(aggregated_result)

In [25]:
# OK, so we need to break out the sub-fields from ID, and we need to multily our total count by x100 since
# we started by only taking every 100th row...

summary_df[['month', 'member_casual', 'rideable_type']] = summary_df['_id'].apply(lambda x: pd.Series([x['month'], x['member_casual'], x['rideable_type']]))
summary_df['total_count'] = summary_df['count']*100
summary_df.drop(['_id', 'count'], axis=1, inplace=True)
summary_df

,avg_distance,avg_duration,month,member_casual,rideable_type,total_count
0,1.945208,12.004025,10,member,classic_bike,2936800
1,1.624319,10.181322,10,member,electric_bike,164100
2,1.464992,9.916469,12,member,electric_bike,67500
3,2.255973,22.349853,5,casual,electric_bike,91000
4,2.186971,23.578372,6,casual,electric_bike,76600
5,1.891798,17.479729,2,casual,classic_bike,149800
6,1.727828,10.145283,1,member,electric_bike,305600
7,2.193758,21.683535,7,casual,classic_bike,753800
8,1.695913,27.613333,7,casual,docked_bike,9500
9,2.133178,19.297766,7,casual,electric_bike,77600


In [26]:
# Ok this will work as a summary data set to start out with for our analysis, to see if we can find any trends
summary_df.to_csv('./data/summary.csv', index=False)

In [ ]:
# One interesting thing I found in plotting the above data (eliminating the "docked_bike" category as not well understood)
# was that the percentage of the total rentals that e-bikes represented appeared to be diminshing.  I'll need another CSV 
# file in order to understand that. 

In [72]:
# First, we need to summarize all rides regardless of rideable type...

group_query = {
    '$group': {
        '_id': {
            'month': '$month'
        },
        'count': { '$sum': 1 },
        'avg_distance': {'$avg': '$distance'},
        'avg_duration': {'$avg': '$ride_duration'}
    },
}

aggregated = list(rides.aggregate([group_query]))

overall_df = pd.DataFrame(aggregated)
overall_df

,_id,count,avg_distance,avg_duration
0,{'month': 3},21869,1.725458,11.621233
1,{'month': 4},28469,1.827075,13.454222
2,{'month': 1},18497,1.687225,11.393077
3,{'month': 8},40811,1.976104,14.062804
4,{'month': 12},22662,1.817961,11.381537
5,{'month': 5},35702,1.855532,14.019112
6,{'month': 6},35517,1.886976,13.885452
7,{'month': 7},37666,1.929607,13.869092
8,{'month': 9},35594,1.943996,13.245964
9,{'month': 10},38081,1.962974,12.975799


In [73]:
overall_df[['month']] = overall_df['_id'].apply(lambda x: pd.Series([x['month']]))
overall_df['count_all_rides'] = overall_df['count']*100
overall_df.drop(['_id', 'count'], axis=1, inplace=True)
overall_df

,avg_distance,avg_duration,month,count_all_rides
0,1.725458,11.621233,3,2186900
1,1.827075,13.454222,4,2846900
2,1.687225,11.393077,1,1849700
3,1.976104,14.062804,8,4081100
4,1.817961,11.381537,12,2266200
5,1.855532,14.019112,5,3570200
6,1.886976,13.885452,6,3551700
7,1.929607,13.869092,7,3766600
8,1.943996,13.245964,9,3559400
9,1.962974,12.975799,10,3808100


In [ ]:
# That gets us a df called `overall_df` that includes the summary across all types
# of rides, separated by month.

In [74]:
# Now we need the same thing, but for e-bikes only...

pipeline = [
    {
    '$match': {'rideable_type': 'electric_bike'}},
    {
    '$group': {
        '_id': {
            'month': '$month',
            'rideable_type': '$rideable_type'
        },
        'count': { '$sum': 1 },
        'avg_distance': {'$avg': '$distance'},
        'avg_duration': {'$avg': '$ride_duration'}
    }
}]

aggregated = list(rides.aggregate(pipeline))

In [75]:
ebike_summary_df = pd.DataFrame(aggregated)

In [76]:
# So this gets us the monthly totals for ebikes only... 

ebike_summary_df[['month', 'rideable_type']] = ebike_summary_df['_id'].apply(lambda x: pd.Series([x['month'], x['rideable_type']]))
ebike_summary_df['ebike_count'] = ebike_summary_df['count']*100
ebike_summary_df.drop(['_id', 'count'], axis=1, inplace=True)
ebike_summary_df

,avg_distance,avg_duration,month,rideable_type,ebike_count
0,1.935711,13.144545,5,electric_bike,447600
1,1.886527,12.861350,6,electric_bike,369300
2,1.771266,12.187538,8,electric_bike,310000
3,1.689741,11.148608,10,electric_bike,205900
4,1.474305,9.975459,11,electric_bike,72600
5,1.505333,12.078185,12,electric_bike,84500
6,1.737299,11.899439,9,electric_bike,249700
7,1.695608,10.891105,2,electric_bike,354300
8,1.837176,12.796677,7,electric_bike,309500
9,1.756528,11.092624,3,electric_bike,442900


In [81]:
# So we just need to merge the ebike data with the overall data to get a dataframe...
ebike_summary_df = pd.merge(ebike_summary_df, overall_df, on='month', how='inner', suffixes=('_df1', '_df2'))
ebike_summary_df=ebike_summary_df[['month', 'rideable_type', 'ebike_count', 'count_all_rides']]
ebike_summary_df

,month,rideable_type,ebike_count,count_all_rides
0,5,electric_bike,447600,3570200
1,6,electric_bike,369300,3551700
2,8,electric_bike,310000,4081100
3,10,electric_bike,205900,3808100
4,11,electric_bike,72600,2878200
5,12,electric_bike,84500,2266200
6,9,electric_bike,249700,3559400
7,2,electric_bike,354300,1746000
8,7,electric_bike,309500,3766600
9,3,electric_bike,442900,2186900


In [82]:
# Ok this will work as an ebike summary
ebike_summary_df.to_csv('./data/ebike_summary.csv', index=False)

In [85]:
# For building our map of rides, we are going to need to export the full dataset...

rides=db['full_data']
pprint(rides.find_one(), indent=2)


{ '_id': '219BF53DA8A2E0C0',
  'distance': 2.3990458227909364,
  'end_lat': 40.877964,
  'end_lng': -73.8847550291,
  'end_station_id': 8795.03,
  'end_station_name': 'Grand Concourse & E Mosholu Pkwy S',
  'ended_at': datetime.datetime(2023, 1, 28, 20, 35, 31),
  'member_casual': 'member',
  'month': 1,
  'ride_duration': 18.9,
  'rideable_type': 'classic_bike',
  'start_lat': 40.860086441,
  'start_lng': -73.900726914,
  'start_station_id': 8550.04,
  'start_station_name': 'Morris Ave & E 184 St',
  'started_at': datetime.datetime(2023, 1, 28, 20, 16, 37)}


In [88]:
rides_df = pd.DataFrame(list(rides.find()))
rides_df.head()

,_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration,distance,month
0,219BF53DA8A2E0C0,classic_bike,2023-01-28 20:16:37,2023-01-28 20:35:31,Morris Ave & E 184 St,8550.04,Grand Concourse & E Mosholu Pkwy S,8795.03,40.860086,-73.900727,40.877964,-73.884755,member,18.900000,2.399046,1
1,F0833A8223F2EEE3,classic_bike,2023-01-31 13:08:23,2023-01-31 13:23:02,Great Jones St,5636.11,Carlton Ave & Flushing Ave,4732.08,40.727510,-73.993965,40.697787,-73.973736,member,14.650000,3.718976,1
2,8ED5CC5F1F8E0A61,classic_bike,2023-01-10 16:36:38,2023-01-10 16:43:49,10 Ave & W 28 St,6459.04,W 20 St & 7 Ave,6182.02,40.750664,-74.001768,40.742388,-73.997262,member,7.183333,0.995453,1
3,735DDD8860E991AE,classic_bike,2023-01-02 11:37:11,2023-01-02 11:42:27,E 22 St & 2 Ave,5971.09,E 33 St & 1 Ave,6197.08,40.737169,-73.981225,40.743227,-73.974498,member,5.266667,0.880325,1
4,8B5B3611B578D341,classic_bike,2023-01-20 16:24:36,2023-01-20 16:34:40,Butler St & Court St,4339.01,Bergen St & Flatbush Ave,4281.08,40.684989,-73.994403,40.680945,-73.975673,member,10.066667,1.642142,1


In [90]:
# So, that's our entire dataset. Let's write it out to a CSV.

rides_df.to_csv('./data/full_data.csv', index=False)

In [109]:
# It's not just full data that we are going to need, though... We will also want destinations by ride type to see if there is any difference I can see
# in terms of popularity of given stations by type of bike...

agg_end_station_id_df = rides_df.groupby(['end_station_id', 'month', 'end_station_name', 'end_lat', 'end_lng', 'rideable_type']).agg({'_id': 'count'}).reset_index()
agg_end_station_id_df['ride_count'] = agg_end_station_id_df['_id']*100
agg_end_station_id_df.drop(['_id'], axis=1, inplace=True)
agg_end_station_id_df.head()

,end_station_id,month,end_station_name,end_lat,end_lng,rideable_type,ride_count
0,2733.03,1,67 St & Erik Pl,40.633385,-74.016562,classic_bike,200
1,2733.03,1,67 St & Erik Pl,40.633385,-74.016562,electric_bike,200
2,2733.03,2,67 St & Erik Pl,40.633385,-74.016562,classic_bike,200
3,2733.03,3,67 St & Erik Pl,40.633385,-74.016562,classic_bike,200
4,2733.03,4,67 St & Erik Pl,40.633385,-74.016562,classic_bike,200


In [110]:
agg_end_station_id_df.to_csv('./data/end_station_id_counts.csv', index=False)

In [112]:
# We may want to build a map where, by destination, we just have the total count by month, not segregated by e-bike, pedal bike,
# or anything else.  Instead, let's just do destination stations by month.

agg_destination_df = rides_df.groupby(['end_station_id', 'month', 'end_station_name', 'end_lat', 'end_lng']).agg({'_id': 'count'}).reset_index()
agg_destination_df['ride_count'] = agg_destination_df['_id']*100
agg_destination_df.drop(['_id'], axis=1, inplace=True)
agg_destination_df.head()

,end_station_id,month,end_station_name,end_lat,end_lng,ride_count
0,2733.03,1,67 St & Erik Pl,40.633385,-74.016562,400
1,2733.03,2,67 St & Erik Pl,40.633385,-74.016562,200
2,2733.03,3,67 St & Erik Pl,40.633385,-74.016562,200
3,2733.03,4,67 St & Erik Pl,40.633385,-74.016562,200
4,2733.03,5,67 St & Erik Pl,40.633385,-74.016562,300


In [113]:
agg_destination_df.to_csv('./data/destination_counts.csv', index=False)